In [1]:
import os
import skimage
from skimage import io, transform
import math
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, concatenate,ZeroPadding2D, BatchNormalization,Concatenate,GlobalAveragePooling2D, Dropout,Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img  
import cv2

%matplotlib inline

import keras.backend as K
K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
def Conv2D_BN_AC(X, filters, kernel, strides, padding, activation):
    X = Conv2D(filters, kernel, strides=strides, padding=padding)(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation(activation)(X)
    return X

def Stem(X):
    X = Conv2D_BN_AC(X, 32, (3, 3), (2, 2), 'valid', 'relu')
    X = Conv2D_BN_AC(X, 32, (3, 3), (1, 1), 'valid', 'relu')
    X = Conv2D_BN_AC(X, 64, (3, 3), (1, 1), 'same', 'relu')
    a = MaxPooling2D((3, 3), strides=(2, 2))(X)
    b = Conv2D_BN_AC(X, 96, (3, 3), (2, 2), 'valid', 'relu')
    X = concatenate([a,b], axis=3)  
    a = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    a = Conv2D_BN_AC(a, 96, (3, 3), (1, 1), 'valid', 'relu')
    b = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(b, 64, (7, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(b, 64, (1, 7), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(b, 96, (3, 3), (1, 1), 'valid', 'relu')
    X = concatenate([a,b], axis=3)
    a = Conv2D_BN_AC(X, 192, (3, 3), (2, 2), 'valid', 'relu')
    b = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = concatenate([a,b], axis=3)
    X = Activation('relu')(X)
    return X

def Inception_resnet_A(X):
    a = Conv2D_BN_AC(X, 32, (1, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(X, 32, (1, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(b, 32, (3, 3), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(X, 32, (1, 1), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 48, (3, 3), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 64, (3, 3), (1, 1), 'same', 'relu')
    con = concatenate([a,b,c], axis=3)
    con = Conv2D_BN_AC(con, 384, (1, 1), (1, 1), 'same', 'linear')
    X = layers.add([X, con])
    X = Activation('relu')(X)
    return X

def Inception_resnet_B(X):
    a =  Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    b =  Conv2D_BN_AC(X, 128, (1, 1), (1, 1), 'same', 'relu')
    b =  Conv2D_BN_AC(b, 160, (1, 7), (1, 1), 'same', 'relu')
    b =  Conv2D_BN_AC(b, 192, (7, 1), (1, 1), 'same', 'relu')
    con = concatenate([a,b], axis=3)
    con = Conv2D_BN_AC(con, 1152, (1, 1), (1, 1), 'same', 'linear')
    X = layers.add([X, con])
    X = Activation('relu')(X)
    return X

def Inception_resnet_C(X):
    a = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(b, 224, (1, 3), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(b, 256, (3, 1), (1, 1), 'same', 'relu')
    con = concatenate([a,b], axis=3)
    con = Conv2D_BN_AC(con, 2144, (1, 1), (1, 1), 'same', 'linear')
    X = layers.add([X, con])
    X = Activation('relu')(X)
    return X

def Reduction_A(X):
    a = MaxPooling2D((3, 3), strides=(2, 2))(X)
    b = Conv2D_BN_AC(X, 384, (3, 3), (2, 2), 'valid', 'relu')
    c = Conv2D_BN_AC(X, 256, (1, 1), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 256, (3, 3), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 384, (3, 3), (2, 2), 'valid', 'relu')
    X = concatenate([a,b,c], axis=3)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    return X

def Reduction_B(X):
    a = MaxPooling2D((3, 3), strides=(2, 2))(X)
    b = Conv2D_BN_AC(X, 256, (1, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(b, 384, (3, 3), (2, 2), 'valid', 'relu')
    c = Conv2D_BN_AC(X, 256, (1, 1), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 288, (3, 3), (2, 2), 'valid', 'relu')
    d = Conv2D_BN_AC(X, 256, (1, 1), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(d, 288, (3, 3), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(d, 320, (3, 3), (2, 2), 'valid', 'relu')
    X = concatenate([a,b,c,d], axis=3)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    return X

def Inception_ResNet_v2(input_shape = (299,299,3)):
    X_input = Input(input_shape)
    X = Stem(X_input)
    for i in np.arange(10):
        X = Inception_resnet_A(X)
    X = Reduction_A(X)
    for i in np.arange(20):
        X = Inception_resnet_B(X)
        
    aux_out = AveragePooling2D((5, 5), strides=(3, 3))(X)
    aux_out = Conv2D_BN_AC(aux_out, 128, (1, 1), (1, 1), 'same', 'relu')
    aux_out = Conv2D_BN_AC(aux_out, 768, (5, 5), (5, 5), 'valid', 'relu')
    aux_out = Flatten()(aux_out)
    aux_out = Dense(2, activation='softmax')(aux_out)
   
    X = Reduction_B(X)    
    for i in np.arange(10):
        X = Inception_resnet_C(X)
    X = AveragePooling2D((8,8))(X)
    X = Dropout(0.8)(X)
    X = Flatten()(X)
    X = Dense(2, activation='softmax')(X)

    model = Model(inputs = X_input, outputs = [X,aux_out], name='Inception_ResNet_v2')
    return model


In [3]:
model = Inception_ResNet_v2()

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

batch_normalization_246 (BatchN (None, 8, 8, 2144)   8576        conv2d_244[0][0]                 
__________________________________________________________________________________________________
conv2d_186 (Conv2D)             (None, 5, 5, 128)    147584      average_pooling2d_1[0][0]        
__________________________________________________________________________________________________
activation_286 (Activation)     (None, 8, 8, 2144)   0           batch_normalization_246[0][0]    
__________________________________________________________________________________________________
batch_normalization_187 (BatchN (None, 5, 5, 128)    512         conv2d_186[0][0]                 
__________________________________________________________________________________________________
add_40 (Add)                    (None, 8, 8, 2144)   0           activation_281[0][0]             
                                                                 activation_286[0][0]             
__________

In [12]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'], loss_weights=[1., 0.3])

In [ ]:
model.fit(x_train, y_train,epochs = 200, batch_size=32,validation_data=(x_test,y_test), shuffle = True)